<a href="https://colab.research.google.com/github/bosunKwak/BigData/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Overall Process
<img width="380" alt="스크린샷 2022-04-19 오후 11 29 44" src="https://user-images.githubusercontent.com/87002218/164027812-736c6c74-e20b-4fcc-a057-dc2c1524d2c2.png">


"here is the example"
1) Tokenize : here, is , the ,example
- 문장을 토큰 시퀀스로 분석하는 과정
- 토큰의 분리기준은 다를 수 있지만 문장을 구성하는 기본단위 -ex) 단어

2) Vocabulary : 475, 21, 2, 30, 5297
- 모은 단어의 토큰들을 합쳐 하나의 list를 만듦(어휘 집합 == 토크나이즈 수행 후 중복을 제거한 토큰들의 집합)
- 각 단어를 문자 단위 ID 시퀀스로 변환하기 위해 사용

3) Embedding : [[0.81,0.27,0.59], [ , , ], [ , , ], [ , , ]]
- 컴퓨터는 인간이 사용하는 자연어를 있는 그대로 이해하는 것이 아니라 숫자를 계산
- 사람이 쓰는 자연어를 기계가 이해할 수 잇는 숫자의 나열인 벡터로 바꾼 결과/ 일련의 과정 전체
- 단어나 문장을 각각 벡터로 변환해 벡터 공간(vector space)로 끼워넣는다(embed) 

# Text classification
- [pytorch를 이용해서 모델을 개발하는 전반적인 과정](https://github.com/bosunKwak/BigData/blob/main/overall_process.ipynb)을 text data를 가지고 실습
- [tutorial 참고](https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html)
- data preparation & model development 
- [dataset](https://pytorch.org/text/stable/datasets.html#ag-news)

## Access to the raw dataset iterators

In [ ]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = iter(AG_NEWS(split='train'))